# **Información climática**

Resolvedor para información climática, obtenido de [la librería fluids](https://fluids.readthedocs.io/tutorial.html#atmospheric-properties).

Versión: 0.0.
Fecha: 2022-07-12.
Realizó: PMB.

Concepto:

Son herramientas para estimar la radiación solar sobre una superficie (directa, difusa, reflejada, etc.)

Mejoras para realizar:
-
Entender la librería [pvlib](https://pvlib-python.readthedocs.io/en/stable/user_guide/clearsky.html).

---

# *Importación de librerías*


En primer lugar, con !pip instalamos los paquetes si Colab no los toma.

In [ ]:
!pip install fluids
!pip install pytz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.7 MB/s eta 0:00:00


Luego, debemos importar las librerías que utilizamos. Algunas con alias, otras sin.

In [ ]:
import fluids as fl
import numpy as np
import pytz
from datetime import datetime, timedelta

# *Ingreso de datos*

Ingresamos los datos de altura sobre el nivel del mar, de latitud y longitud del sitio, de la turbidez de linke (1 para cielo limpio sin polución, y despejado) y el ángulo de tilt de la superficie, que entendemos que es 0° para una superficie horizontal.

Para concluir la carga de datos, debemos dar o bien variables intensivas, o bien variables extensivas. Elegimos el caudal de combustible y el exceso de aire utilizado.

In [ ]:
z = 4832.5 # height
lat = -24.19206
lon = -66.47483
linke_turbidity = 1
surface_tilt = 0 # Si es 90°, entiendo que es una superficie vertical
surface_azimuth = 0
albedo = 0.4

# *Cálculo*

Obtenemos una presión atmosférica promedio para una determinada altura sobre el nivel del mar.

In [ ]:
atm = fl.ATMOSPHERE_1976(Z=z)
pressure = atm.P
print(pressure / 1000) # en kPa(a)

55.26706281956152


Ahora lo que hacemos es usar un modelo más detallado, en el cual recorremos desde el día 1 (ojo que Python es zero-indented) hasta el 365 la presión atmosférica, y luego la promediamos.

In [ ]:
p = np.zeros(365)
for j in range(len(p)):
  atm = fl.ATMOSPHERE_NRLMSISE00(Z=z, latitude=lat, longitude=lon, day=1+j)
  p[j] = atm.P
pressure = np.mean(p) # en Pa(a)
print(pressure / 1000) # en kPa(a)

56.07124267708999


Ahora calculamos la irradiancia solar.

El primer valor que se retorna es la irradiación solar que incide sobre el objeto, en W/m2. Los siguientes dos valores son las componentes de esta radiación, los cuales provienen 1) directamente desde el sol y 2) difusamente, luego de reflejarse desde algún otro objeto. Los dos valores finales parten a este último valor (luz difusa) entre 3) una componente reflejada únicamente en el cielo y las nubes y 4) una componente ocasionada por el albedo terrestre, rebotando desde la superficie, luego el cielo, antes de incidir sobre el objeto -la superficie.

In [ ]:
for j in range(12):
  Sol_Irr = fl.solar_irradiation(
                               Z=z,
                               latitude=lat,
                               longitude=lon,
                               linke_turbidity=linke_turbidity,
                               moment=pytz.timezone('America/Argentina/Salta').localize(datetime(2022, j+1, 1, 12, 0, 0)),
                               surface_tilt=surface_tilt,
                               surface_azimuth=surface_azimuth,
                               albedo=albedo
                               )
  print(Sol_Irr[1])
# pytz.all_timezones ---> con esto sabemos la lista de zonas.
#print(Sol_Irr)

1231.0938015229372
1197.8514464688333
1151.3079585065896
1060.6314715038307
940.995873027644
834.9338284787458
808.4582747223319
884.1854157457058
1028.4819520320361
1163.9967803502684
1243.0035843164442
1254.4289665208355


Con esto saco la irradiación directa a la horizontal, y la comparo contra la que dice HAP para Antofagasta, y con eso saco el factor por el que tengo que multiplicar a lo de HAP.